In [2]:
## All Imports

from IPython.display import HTML
import numpy as np
#import urllib2
import bs4  #Beautiful Soup
import time
import operator
import socket
#import cPickle
import re

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

#import seaborn as sns
#sns.set_context("talk")
#sns.set_style("white")

#from secret import *

In [3]:
#Might need API Registrations for Rotten Tomatoes, Twitter
#Rotten tomatoes have stopped registration
#Will register as examples require it

In [6]:
#Reading data from Movielens under group lens

#Reading users data

# pass the column names for each column in the csv
u_cols = ["user_id","age","sex","occupation","zip_code"]

#Read the u.user file from group lens database
users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user',
    sep = '|', names = u_cols)

#top 5 rows from users table
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
#Read the Ratings data

#names for the columns for the table
r_cols = ["user_id", "movie_id","rating","unix_timestamp"]

#read the rating files; note that the seperator is tab for this file
rating = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.data",
    sep ="\t", names = r_cols)

#print the first 5 ratings
rating.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
#Data about the movies

#The movies file contains more than 5 columns, but we are only considering
# the first five columns

#Naming the first five columns
m_cols =["movie_id","title","release_date","video_release_date","imdb_url"]

movies = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.item",
    sep = "|", names = m_cols, usecols = range(5)) #usecols considers columns
                                                #specified
#print the first 5 movies
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


In [15]:
#Getting some information about the data
print(movies.dtypes) #Gives information about the content of each column
                     # as interpreted by python
print()
print(movies.describe()) #.describe find the columns that are numerical
                        # (and not objects or complex) and provides general
                        # statistics about it.
#Note the only columns thatare chose are movie_id = int64 and video_release
#_date that is float 64

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object

          movie_id  video_release_date
count  1682.000000                 0.0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


In [23]:
#Selecting specific data from a set

users.head() #Just to see what data is in users
print(users["occupation"].head()) #just first five occupation
print()
#print specific columns
modtable = users[["occupation","sex"]] #note enty insider users is a list
print(modtable.head())
print()
newcol = ["occupation","sex"]
modtable2 = users[newcol]
print(modtable2.head())

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object

   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F

   occupation sex
0  technician   M
1       other   F
2      writer   M
3  technician   M
4       other   F


In [31]:
#Filtering data

#select data for users older than 25
above25 = users["age"]>25
print(users[above25].head())
print()
#Another way
print(users[users.age>25].head())
print()
users[users.age>25].head() #Not sure if the output format difference
                            # are worth looking, but fyi



   user_id  age sex     occupation zip_code
1        2   53   F          other    94043
4        5   33   F          other    15213
5        6   42   M      executive    98101
6        7   57   M  administrator    91344
7        8   36   M  administrator    05201

   user_id  age sex     occupation zip_code
1        2   53   F          other    94043
4        5   33   F          other    15213
5        6   42   M      executive    98101
6        7   57   M  administrator    91344
7        8   36   M  administrator    05201



,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


In [41]:
#Find users above 40 and are male
users.dtypes
users[(users.age>40) & (users.sex=="M")].head()

,user_id,age,sex,occupation,zip_code
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
9,10,53,M,lawyer,90703
12,13,47,M,educator,29206
13,14,45,M,scientist,55106


In [44]:
#show the mean age for women programmers
print(users[(users.sex == "F") & (users.occupation == "programmer")].describe)
print()
print(users[(users["sex"]=="F") & (users["occupation"]=="programmer")].mean())
print()
print(users[(users.sex=="F") & (users.occupation=="programmer")].mean())

<bound method NDFrame.describe of      user_id  age sex  occupation zip_code
291      292   35   F  programmer    94703
299      300   26   F  programmer    55106
351      352   37   F  programmer    55105
403      404   29   F  programmer    55108
420      421   38   F  programmer    55105
697      698   28   F  programmer    06906>

user_id    411.166667
age         32.166667
dtype: float64

user_id    411.166667
age         32.166667
dtype: float64


In [ ]:
#Split and combine:
# Splitting the groups into data based on criteria
# applying a common function to each group independently
# combining the data


#Find the users who provided a lot of ratings:

#Method:
    # Split data per user ID
    # Count the ratings
    # combine the result